In [52]:
import os
import cv2
from glob import glob
from ipywidgets import interact
import copy
from sahi.utils.cv import Colors
import matplotlib.pyplot as plt
os.getcwd()

'/home/yyj/sahi/demo'

## YOLO Label 기반

In [53]:
VIEW_CLASS_ID = [2]
LABEL_DIR = "../resources/log/labels/"
IMAGE_DIR = "../resources/log/images/"

In [54]:
label_files = sorted(glob(LABEL_DIR+"*.txt"))
len(label_files)

1533

In [55]:
choose_labels = []

In [56]:
for label_file in label_files:
    with open(label_file, 'r') as f:
        line = f.readline().strip()
        while line:
            id, cx, cy, w, h = list(map(float, line.split()))
            if id in VIEW_CLASS_ID:
                choose_labels.append(label_file)
                break
            line = f.readline().strip()

In [57]:
CLASS_NAME_TO_ID = {'Buoy': 0, 'Boat': 1, 'Channel Marker': 2, 'Speed Warning Sign': 3}
CLASS_ID_TO_NAME = {0: 'Buoy', 1: 'Boat', 2: 'Channel Marker', 3: 'Speed Warning Sign'}
SIZE=(1920,1080)

def visualize_yolo(image, bboxes, category_ids, color=None):
    image = copy.deepcopy(image)
    
    if color is None:
        colors = Colors()
    else:
        colors = None
        
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = CLASS_ID_TO_NAME[category_id[0]]
        
        x_center, y_center, w, h = bbox
        x_min = int((x_center - w/2) * SIZE[0])
        y_min = int((y_center - h/2) * SIZE[1])
        x_max = int((x_center + w/2) * SIZE[0])
        y_max = int((y_center + h/2) * SIZE[1])

        if colors is not None:
            color = colors(category_id[0])
        
        # set rect_th for boxes
        rect_th = max(round(sum(image.shape) / 2 * 0.001), 1)
        # set text_th for category names
        text_th = max(rect_th - 1, 1)
        # set text_size for category names
        text_size = rect_th / 3

        cv2.rectangle(image, 
                      (x_min, y_min),
                      (x_max, y_max), 
                      color=color,
                      thickness=rect_th)
        
        p1, p2 = (int(x_min), int(y_min)), (int(x_max), int(y_max))
        label = f"{class_name}"
        w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            image,
            label,
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0,
            text_size,
            (255, 255, 255),
            thickness=text_th,
        )
        
    return image

In [58]:
@interact(index=(0,len(choose_labels)))
def verify_gt(index=0):
    label = choose_labels[index]
    image_name = os.path.join(IMAGE_DIR, os.path.splitext(os.path.basename(label))[0]+".jpg")
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    bboxes = []
    category_ids = []
    
    with open(label, 'r') as f:
        line = f.readline().strip()
        while line:
            id, cx, cy, w, h = list(map(float, line.split()))
            bboxes.append([cx, cy, w, h])
            category_ids.append([int(id)])
            line = f.readline().strip()

    if len(bboxes) > 0:    
        canvas = visualize_yolo(image, bboxes, category_ids)
        plt.figure(figsize=(16,16))
        plt.imshow(canvas)
        plt.axis('off')
        plt.show()

interactive(children=(IntSlider(value=0, description='index', max=506), Output()), _dom_classes=('widget-inter…

## Save specific frame with button

In [59]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
    
button = widgets.Button(description='My Button')
out = widgets.Output()

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    def on_button_clicked(_):
        # "linking function with output"
        with out:
            # what happens when we press the button
            clear_output()
            
    # linking button and function together using a button's method
    button.on_click(on_button_clicked)
    # displaying button and its output together
   
    
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

widgets.VBox([button,out])

NameError: name 'image_files' is not defined

In [ ]:
@interact(index=(0,len(choose_labels)))
def verify_gt(index=0):
    label = choose_labels[index]
    image_name = os.path.join(IMAGE_DIR, os.path.splitext(os.path.basename(label))[0]+".jpg")
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    bboxes = []
    category_ids = []
    
    with open(label, 'r') as f:
        line = f.readline().strip()
        while line:
            id, cx, cy, w, h = list(map(float, line.split()))
            bboxes.append([cx, cy, w, h])
            category_ids.append([int(id)])
            line = f.readline().strip()

    if len(bboxes) > 0:    
        canvas = visualize_yolo(image, bboxes, category_ids)
        plt.figure(figsize=(16,16))
        plt.imshow(canvas)
        plt.axis('off')
        plt.show()

## JSON 기반